# FINMA LCR Monitoring Dashboard
## Liquidity Coverage Ratio - Real-Time Compliance Monitoring

**Regulatory Basis**: FINMA Circular 2015/2, LiqV Art. 14-20, Basel III LCR Framework  
**Reporting Frequency**: Daily (60-minute refresh)  
**Compliance Minimum**: 100% LCR  
**Internal Target**: 110% LCR  

---

## Overview

This dashboard provides **real-time monitoring** of the Liquidity Coverage Ratio (LCR) for compliance with Swiss regulatory requirements. The LCR measures a bank's ability to survive a 30-day stressed liquidity scenario.

### The LCR Formula

```
LCR = (High-Quality Liquid Assets / Net Cash Outflows over 30 days) × 100%
```

**Compliance Thresholds**:
- **≥ 100%**: PASS (Fully compliant)
- **95-99%**: WARNING (Immediate action required)
- **< 95%**: FAIL (Critical breach - report to FINMA within 24 hours)

---

## Dashboard Sections

1. **Current LCR Status** - Executive summary with key metrics
2. **90-Day Trend Analysis** - Historical performance and volatility
3. **HQLA Composition** - Asset portfolio breakdown by level (L1/L2A/L2B)
4. **Deposit Outflows** - Stressed cash outflows by counterparty type
5. **Compliance Alerts** - Active alerts and monthly summary for SNB reporting

---

In [ ]:
# Import required libraries
import streamlit as st
import pandas as pd
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

st.info(f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}")

In [ ]:
# Setup: Initialize Snowflake session
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('REP_AGG_001')
st.success('Connected to LCR Monitoring Platform')

---

## Section 1: Current LCR Status

**Executive Summary** - Real-time liquidity position with color-coded compliance status

In [ ]:
# Query current LCR status from semantic view
df_current = session.sql("""
    SELECT * FROM LCRS_AGG_VW_LCR_CURRENT
""").to_pandas()

# Extract key metrics
lcr = df_current['LCR_RATIO_PCT'].iloc[0]
status = df_current['COMPLIANCE_STATUS'].iloc[0]
severity = df_current['STATUS_COLOR'].iloc[0]
hqla = df_current['HQLA_TOTAL_CHF'].iloc[0] / 1e9  # Convert to billions
outflow = df_current['NET_CASH_OUTFLOWS_CHF'].iloc[0] / 1e9
buffer_chf = df_current['LIQUIDITY_BUFFER_CHF'].iloc[0] / 1e9
buffer_pct = df_current['BUFFER_PCT_OF_OUTFLOWS'].iloc[0]
cap_applied = df_current['IS_40PCT_CAP_APPLIED'].iloc[0]
as_of_date = df_current['REPORTING_DATE'].iloc[0]

# Color coding for visualizations
color_map = {
    'GREEN': 'green',
    'YELLOW': 'orange',
    'RED': 'red',
    'GRAY': 'gray'
}
color = color_map.get(severity, 'gray')

# Display LCR metrics using Streamlit
st.header(f"Current LCR Status (as of {as_of_date})")

# Status indicator with color
if severity == 'RED':
    st.error(f"Status: {status} - CRITICAL BREACH")
elif severity == 'YELLOW':
    st.warning(f"Status: {status} - WARNING")
else:
    st.success(f"Status: {status} - COMPLIANT")

# Create columns for key metrics
col1, col2, col3, col4 = st.columns(4)

with col1:
    st.metric(
        label="LCR Ratio",
        value=f"{lcr:.1f}%",
        delta=f"{lcr - 110:.1f}% vs target"
    )
    
with col2:
    st.metric(
        label="Total HQLA",
        value=f"CHF {hqla:.2f}B"
    )
    
with col3:
    st.metric(
        label="Net Outflows",
        value=f"CHF {outflow:.2f}B"
    )
    
with col4:
    st.metric(
        label="Liquidity Buffer",
        value=f"CHF {buffer_chf:.2f}B",
        delta=f"+{buffer_pct:.1f}%"
    )

# Additional details
st.subheader("Portfolio Composition")
col5, col6, col7 = st.columns(3)

with col5:
    st.metric("40% Cap Status", "APPLIED" if cap_applied else "COMPLIANT")
with col6:
    st.metric("Level 1 HQLA", f"{df_current['LEVEL1_PCT_OF_HQLA'].iloc[0]:.1f}%")
with col7:
    st.metric("Level 2 HQLA", f"{df_current['LEVEL2_PCT_OF_HQLA'].iloc[0]:.1f}%")

# Assessment commentary
if lcr >= 110:
    st.info("[STRONG] LCR significantly above regulatory minimum. Comfortable liquidity buffer.")
elif lcr >= 100:
    st.info("[PASS] LCR meets regulatory minimum. Monitor closely and maintain buffer.")
elif lcr >= 95:
    st.warning("[WARNING] LCR below internal target. Immediate liquidity sourcing required.")
else:
    st.error("[CRITICAL] LCR breach! Report to FINMA within 24 hours. Emergency measures required.")

---

## Section 2: 90-Day Trend Analysis

**Historical Performance** - LCR trend with moving averages and volatility analysis

In [ ]:
# Query 90-day trend data
df_trend = session.sql("""
    SELECT * FROM LCRS_AGG_VW_TREND_90DAY
    ORDER BY REPORTING_DATE
""").to_pandas()

# Display trend chart using Streamlit
st.header("90-Day LCR Trend Analysis")

# Prepare data for line chart
chart_data = df_trend[['REPORTING_DATE', 'LCR_RATIO_PCT', 'MA7_LCR_RATIO', 'MA30_LCR_RATIO']].set_index('REPORTING_DATE')
chart_data.columns = ['Daily LCR', '7-Day MA', '30-Day MA']

# Display the line chart
st.line_chart(chart_data)

# Add reference lines information
col1, col2 = st.columns(2)
with col1:
    st.info("Regulatory Minimum: 100%")
with col2:
    st.success("Internal Target: 110%")

# Highlight breaches if any
breach_periods = df_trend[df_trend['IS_BREACH'] == True]
if not breach_periods.empty:
    st.error(f"Warning: {len(breach_periods)} breach days detected (LCR < 100%)")
    st.dataframe(breach_periods[['REPORTING_DATE', 'LCR_RATIO_PCT']], hide_index=True)

# Statistical summary
print(f"\n{'='*80}")
print(f"90-DAY TREND SUMMARY")
print(f"{'='*80}\n")

print(f"{'Current LCR:':<30} {df_trend['LCR_RATIO_PCT'].iloc[-1]:>8.1f}%")
print(f"{'Average LCR:':<30} {df_trend['LCR_RATIO_PCT'].mean():>8.1f}%")
print(f"{'Minimum LCR:':<30} {df_trend['LCR_RATIO_PCT'].min():>8.1f}%")
print(f"{'Maximum LCR:':<30} {df_trend['LCR_RATIO_PCT'].max():>8.1f}%")
print(f"{'Std Deviation:':<30} {df_trend['LCR_RATIO_PCT'].std():>8.1f}% (volatility)")
print(f"{'-'*80}")

breach_count = df_trend['IS_BREACH'].sum()
warning_count = df_trend['IS_WARNING'].sum() - breach_count

print(f"{'Days < 100% (Breach):':<30} {breach_count:>8.0f} days")
print(f"{'Days 100-105% (Warning):':<30} {warning_count:>8.0f} days")
print(f"{'Days > 105% (Compliant):':<30} {len(df_trend) - warning_count:>8.0f} days")
print(f"{'-'*80}")

# Volatility analysis
volatile_days = df_trend[df_trend['VOLATILITY_RATING'].isin(['High', 'Very High'])]
print(f"{'High Volatility Days:':<30} {len(volatile_days):>8.0f} days")
print(f"{'Largest Daily Change:':<30} {df_trend['DAY_OVER_DAY_CHANGE_PCT'].abs().max():>8.1f}%")
print(f"{'Average Daily Change:':<30} {df_trend['DAY_OVER_DAY_CHANGE_PCT'].abs().mean():>8.1f}%")
print(f"{'='*80}\n")

# Trend assessment
recent_trend = df_trend['LCR_RATIO_PCT'].iloc[-7:].mean()
older_trend = df_trend['LCR_RATIO_PCT'].iloc[-30:-7].mean()

if recent_trend > older_trend + 2:
    print("[IMPROVING] LCR has been strengthening over the last 7 days.")
elif recent_trend < older_trend - 2:
    print("[DECLINING] LCR has been weakening over the last 7 days. Monitor closely.")
else:
    print("[STABLE] LCR has remained relatively stable over the last 7 days.")


---

## Section 3: HQLA Composition Breakdown

**Asset Portfolio** - High-Quality Liquid Assets by level (L1/L2A/L2B) and asset type

In [ ]:
# Query HQLA breakdown from semantic view
df_hqla = session.sql("""
    SELECT * FROM LCRS_AGG_VW_HQLA_BREAKDOWN
    ORDER BY LEVEL_SORT_ORDER, HQLA_VALUE_CHF DESC
""").to_pandas()

# Calculate billions for easier reading
df_hqla['HQLA_VALUE_BILLIONS'] = df_hqla['HQLA_VALUE_CHF'] / 1e9
df_hqla['AVG_VALUE_PER_HOLDING_CHF'] = df_hqla['MARKET_VALUE_CHF'] / df_hqla['HOLDINGS_COUNT']

# Display HQLA composition using Streamlit
st.header('HQLA Composition Breakdown')

# Create pivot table for stacked bar chart
pivot_hqla = df_hqla.pivot(index='ASSET_LEVEL', columns='ASSET_TYPE', values='HQLA_VALUE_BILLIONS').fillna(0)

# Display stacked bar chart
st.bar_chart(pivot_hqla)

# Display detailed breakdown table
print(f"\n{'='*80}")
print(f"HQLA COMPOSITION BREAKDOWN")
print(f"{'='*80}\n")

# Group by level
for level in ['Level 1', 'Level 2A', 'Level 2B']:
    level_data = df_hqla[df_hqla['ASSET_LEVEL'] == level]
    if not level_data.empty:
        total = level_data['HQLA_VALUE_BILLIONS'].sum()
        pct = level_data['PCT_OF_TOTAL_HQLA'].sum()
        print(f"\n{level} (Total: CHF {total:.2f}B, {pct:.1f}% of HQLA)")
        print(f"{'-'*80}")
        
        for _, row in level_data.iterrows():
            print(f"  {row['ASSET_TYPE']:<35} CHF {row['HQLA_VALUE_BILLIONS']:>8.2f}B ({row['PCT_OF_TOTAL_HQLA']:>5.1f}%)")
            print(f"    Haircut: {row['HAIRCUT_PCT']:>3.0f}%  |  Holdings: {row['HOLDINGS_COUNT']:>3.0f}  |  Avg per holding: CHF {row['AVG_VALUE_PER_HOLDING_CHF']/1e6:>6.1f}M")

print(f"\n{'='*80}")
print(f"{'Total HQLA:':<40} CHF {df_hqla['HQLA_VALUE_BILLIONS'].sum():>8.2f}B")
print(f"{'Total Holdings Count:':<40} {df_hqla['HOLDINGS_COUNT'].sum():>12.0f}")
print(f"{'='*80}\n")

# Quality assessment
level1_pct = df_hqla[df_hqla['ASSET_LEVEL'] == 'Level 1']['PCT_OF_TOTAL_HQLA'].sum()
level2_pct = df_hqla[df_hqla['ASSET_LEVEL'].isin(['Level 2A', 'Level 2B'])]['PCT_OF_TOTAL_HQLA'].sum()

if level2_pct > 40:
    print(f"[WARNING] Level 2 assets at {level2_pct:.1f}% exceed 40% cap. Cap will be applied.")
elif level2_pct > 35:
    print(f"[CAUTION] Level 2 assets at {level2_pct:.1f}% approaching 40% cap. Monitor closely.")
else:
    print(f"[COMPLIANT] Level 2 assets at {level2_pct:.1f}% well within 40% cap limit.")


---

## Section 4: Deposit Outflow Analysis

**Funding Stability** - 30-day stressed net cash outflows by counterparty type and run-off rates

In [ ]:
# Query deposit outflow breakdown from semantic view
df_outflow = session.sql("""
    SELECT * FROM LCRS_AGG_VW_OUTFLOW_BREAKDOWN
    ORDER BY OUTFLOW_AMOUNT_CHF DESC
""").to_pandas()

# Calculate billions for easier reading
df_outflow['DEPOSIT_BALANCE_BILLIONS'] = df_outflow['DEPOSIT_BALANCE_CHF'] / 1e9
df_outflow['OUTFLOW_AMOUNT_BILLIONS'] = df_outflow['OUTFLOW_AMOUNT_CHF'] / 1e9

# Create waterfall chart showing deposit balances → stressed outflows
# Display deposit outflows using Streamlit
st.header('Deposit Outflow Analysis')

# Display outflow metrics
for _, row in df_outflow.iterrows():
    with st.expander(f"{row['COUNTERPARTY_TYPE']} - {row['OUTFLOW_AMOUNT_BILLIONS']:.2f}B CHF"):
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric('Deposit Balance', f"{row['DEPOSIT_BALANCE_BILLIONS']:.2f}B CHF")
        with col2:
            st.metric('Run-off Rate', f"{row['AVG_RUN_OFF_RATE_PCT']:.1f}%")
        with col3:
            st.metric('Stressed Outflow', f"{row['OUTFLOW_AMOUNT_BILLIONS']:.2f}B CHF")

# Bar chart comparison
comparison_data = df_outflow[['COUNTERPARTY_TYPE', 'DEPOSIT_BALANCE_BILLIONS', 'OUTFLOW_AMOUNT_BILLIONS']].set_index('COUNTERPARTY_TYPE')
comparison_data.columns = ['Deposit Balance (B)', 'Stressed Outflow (B)']
st.bar_chart(comparison_data)

# Create stacked bar chart: deposits vs outflows

# Display detailed breakdown table
print(f"\n{'='*80}")
print(f"DEPOSIT OUTFLOW ANALYSIS")
print(f"{'='*80}\n")

print(f"{'Counterparty Type':<25} {'Deposits':>12} {'Run-off':>8} {'Outflow':>12} {'% Total':>8}")
print(f"{'-'*80}")

for _, row in df_outflow.iterrows():
    print(f"{row['COUNTERPARTY_TYPE']:<25} "
          f"CHF {row['DEPOSIT_BALANCE_BILLIONS']:>7.2f}B "
          f"{row['AVG_RUN_OFF_RATE_PCT']:>7.1f}% "
          f"CHF {row['OUTFLOW_AMOUNT_BILLIONS']:>7.2f}B "
          f"{row['PCT_OF_TOTAL_OUTFLOWS']:>7.1f}%")

print(f"{'-'*80}")
print(f"{'TOTAL':<25} "
      f"CHF {df_outflow['DEPOSIT_BALANCE_BILLIONS'].sum():>7.2f}B "
      f"{df_outflow['OUTFLOW_AMOUNT_BILLIONS'].sum() / df_outflow['DEPOSIT_BALANCE_BILLIONS'].sum() * 100:>7.1f}% "
      f"CHF {df_outflow['OUTFLOW_AMOUNT_BILLIONS'].sum():>7.2f}B "
      f"{df_outflow['PCT_OF_TOTAL_OUTFLOWS'].sum():>7.1f}%")
print(f"{'='*80}\n")

# Funding stability assessment
retail_outflow = df_outflow[df_outflow['COUNTERPARTY_TYPE'].str.contains('Retail', case=False, na=False)]['PCT_OF_TOTAL_OUTFLOWS'].sum()
corporate_outflow = df_outflow[df_outflow['COUNTERPARTY_TYPE'].str.contains('Corporate', case=False, na=False)]['PCT_OF_TOTAL_OUTFLOWS'].sum()
wholesale_outflow = df_outflow[df_outflow['COUNTERPARTY_TYPE'].str.contains('Financial|Wholesale', case=False, na=False)]['PCT_OF_TOTAL_OUTFLOWS'].sum()

print(f"FUNDING STABILITY ANALYSIS:")
print(f"{'-'*80}")
print(f"Retail Funding:      {retail_outflow:>5.1f}% of outflows (STABLE - low run-off)")
print(f"Corporate Funding:   {corporate_outflow:>5.1f}% of outflows (MODERATE - medium run-off)")
print(f"Wholesale Funding:   {wholesale_outflow:>5.1f}% of outflows (VOLATILE - high run-off)")
print(f"{'-'*80}\n")

if wholesale_outflow > 40:
    print("[WARNING] High reliance on wholesale funding (>40%). Consider diversification.")
elif wholesale_outflow > 30:
    print("[CAUTION] Moderate wholesale funding exposure (>30%). Monitor funding mix.")
else:
    print("[STABLE] Balanced funding profile with strong retail base.")


---

## Section 5: Compliance Alerts & Monthly Summary

**Regulatory Reporting** - Active alerts and monthly LCR summary for SNB reporting

In [ ]:
# Query active alerts from semantic view
df_alerts = session.sql("""
    SELECT * FROM LCRS_AGG_VW_ALERTS_ACTIVE
    ORDER BY SEVERITY_PRIORITY ASC, ALERT_DATE DESC
""").to_pandas()

# Display alert status
print(f"\n{'='*80}")
print(f"COMPLIANCE ALERTS STATUS")
print(f"{'='*80}\n")

if len(df_alerts) == 0:
    print("[OK] NO ACTIVE ALERTS - LCR compliance is within acceptable parameters.\n")
else:
    print(f"[ALERT] {len(df_alerts)} ACTIVE COMPLIANCE ALERT(S)\n")
    print(f"{'-'*80}\n")
    
    for idx, alert in df_alerts.iterrows():
        severity_label = f"[{alert['SEVERITY']}]"
        print(f"{severity_label:15} {alert['ALERT_TYPE']}")
        print(f"{'Date:':<15} {alert['ALERT_DATE']}")
        print(f"{'LCR Ratio:':<15} {alert['CURRENT_LCR_RATIO']:.1f}%")
        print(f"{'Description:':<15} {alert['DESCRIPTION']}")
        print(f"{'Action:':<15} {alert['RECOMMENDED_ACTION']}")
        print(f"{'-'*80}\n")

print(f"{'='*80}\n")

# Query monthly summary for regulatory reporting
df_monthly = session.sql("""
    SELECT * FROM REPP_AGG_VW_LCR_MONTHLY_SUMMARY
    ORDER BY REPORTING_MONTH DESC
    LIMIT 12
""").to_pandas()

# Display monthly summary table
print(f"\n{'='*80}")
print(f"MONTHLY LCR SUMMARY (Last 12 Months)")
print(f"SNB Regulatory Reporting Preview")
print(f"{'='*80}\n")

print(f"{'Month':<12} {'Days':>6} {'Min%':>8} {'Avg%':>8} {'Max%':>8} {'Breach':>8} {'Status':<15}")
print(f"{'-'*80}")

for _, row in df_monthly.iterrows():
    status = "[BREACH]" if row['BREACH_DAYS'] > 0 else "[COMPLIANT]"
    print(f"{str(row['REPORTING_MONTH'])[:7]:<12} "
          f"{row['TRADING_DAYS']:>6.0f} "
          f"{row['LCR_MIN']:>8.1f} "
          f"{row['LCR_AVG']:>8.1f} "
          f"{row['LCR_MAX']:>8.1f} "
          f"{row['BREACH_DAYS']:>8.0f} "
          f"{status:<15}")

print(f"{'='*80}\n")

# Calculate year-to-date statistics
ytd_data = df_monthly[df_monthly['REPORTING_MONTH'].astype(str).str.startswith(str(datetime.now().year))]
if not ytd_data.empty:
    print(f"YEAR-TO-DATE SUMMARY ({datetime.now().year}):")
    print(f"{'-'*80}")
    print(f"{'Average LCR:':<30} {ytd_data['LCR_AVG'].mean():>8.1f}%")
    print(f"{'Minimum LCR:':<30} {ytd_data['LCR_MIN'].min():>8.1f}%")
    print(f"{'Maximum LCR:':<30} {ytd_data['LCR_MAX'].max():>8.1f}%")
    print(f"{'Total Breach Days:':<30} {ytd_data['BREACH_DAYS'].sum():>8.0f}")
    print(f"{'Total Trading Days:':<30} {ytd_data['TRADING_DAYS'].sum():>8.0f}")
    print(f"{'Breach Rate:':<30} {ytd_data['BREACH_DAYS'].sum() / ytd_data['TRADING_DAYS'].sum() * 100:>8.2f}%")
    print(f"{'='*80}\n")

# Export monthly summary to CSV for SNB submission
export_filename = f"LCR_SNB_Report_{datetime.now().strftime('%Y%m%d')}.csv"
df_monthly.to_csv(export_filename, index=False)
print(f"[EXPORT] Monthly summary exported to: {export_filename}")
print(f"         Ready for SNB submission and regulatory review.\n")

# Final assessment
total_breach_days = df_monthly['BREACH_DAYS'].sum()
total_days = df_monthly['TRADING_DAYS'].sum()

if total_breach_days == 0:
    print("[COMPLIANT] No LCR breaches in the last 12 months. Excellent liquidity management.")
elif total_breach_days / total_days < 0.05:
    print("[MOSTLY_COMPLIANT] Minor breaches detected (<5% of days). Review root causes.")
else:
    print(f"[ATTENTION] {total_breach_days} breach days ({total_breach_days/total_days*100:.1f}%). Immediate action required.")